In [1]:
import pandas as pd
import os
import numpy as np
import re
import requests
import json
from bs4 import BeautifulSoup

In [2]:
character_deaths= pd.read_csv("pipeline-project/inputs/character-deaths.csv")

In [3]:
character_deaths


,Name,Allegiances,Death Year,Book of Death,Death Chapter,Book Intro Chapter,Gender,Nobility,GoT,CoK,SoS,FfC,DwD
0,Addam Marbrand,Lannister,NaN,NaN,NaN,56.0,1,1,1,1,1,1,0
1,Aegon Frey (Jinglebell),None,299.0,3.0,51.0,49.0,1,1,0,0,1,0,0
2,Aegon Targaryen,House Targaryen,NaN,NaN,NaN,5.0,1,1,0,0,0,0,1
3,Adrack Humble,House Greyjoy,300.0,5.0,20.0,20.0,1,1,0,0,0,0,1
4,Aemon Costayne,Lannister,NaN,NaN,NaN,NaN,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,Zollo,None,NaN,NaN,NaN,21.0,1,0,0,0,1,0,0
913,Yurkhaz zo Yunzak,None,300.0,5.0,59.0,47.0,1,0,0,0,0,0,1
914,Yezzan Zo Qaggaz,None,300.0,5.0,57.0,25.0,1,1,0,0,0,0,1
915,Torwynd the Tame,Wildling,300.0,5.0,73.0,73.0,1,0,0,0,1,0,0


In [4]:
death_set = character_deaths.copy()

In [5]:
death_set

,Name,Allegiances,Death Year,Book of Death,Death Chapter,Book Intro Chapter,Gender,Nobility,GoT,CoK,SoS,FfC,DwD
0,Addam Marbrand,Lannister,NaN,NaN,NaN,56.0,1,1,1,1,1,1,0
1,Aegon Frey (Jinglebell),None,299.0,3.0,51.0,49.0,1,1,0,0,1,0,0
2,Aegon Targaryen,House Targaryen,NaN,NaN,NaN,5.0,1,1,0,0,0,0,1
3,Adrack Humble,House Greyjoy,300.0,5.0,20.0,20.0,1,1,0,0,0,0,1
4,Aemon Costayne,Lannister,NaN,NaN,NaN,NaN,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,Zollo,None,NaN,NaN,NaN,21.0,1,0,0,0,1,0,0
913,Yurkhaz zo Yunzak,None,300.0,5.0,59.0,47.0,1,0,0,0,0,0,1
914,Yezzan Zo Qaggaz,None,300.0,5.0,57.0,25.0,1,1,0,0,0,0,1
915,Torwynd the Tame,Wildling,300.0,5.0,73.0,73.0,1,0,0,0,1,0,0


In [6]:
death_set = death_set.drop(["Nobility", "GoT", "CoK", "SoS", "FfC", "DwD"], axis=1)

In [7]:
death_set.tail(60)

,Name,Allegiances,Death Year,Book of Death,Death Chapter,Book Intro Chapter,Gender
857,Viserys Targaryen,House Targaryen,298.0,1.0,46.0,3.0,1
858,Vylarr,House Lannister,NaN,NaN,NaN,3.0,1
859,Vyman,Tully,NaN,NaN,NaN,55.0,1
860,Waif,None,NaN,NaN,NaN,6.0,0
861,Walda Rivers (daughter of Aemon),None,NaN,NaN,NaN,49.0,0
862,Walder Frey,None,NaN,NaN,NaN,28.0,1
863,Walder Rivers,None,NaN,NaN,NaN,35.0,1
864,Wallen,Wildling,298.0,1.0,37.0,37.0,1
865,Walton,None,NaN,NaN,NaN,64.0,1
866,Wat (orphan),None,NaN,NaN,NaN,37.0,1


In [8]:
death_set.columns = death_set.columns.str.strip().str.lower().str.replace(' ', '_')

In [9]:
death_set

,name,allegiances,death_year,book_of_death,death_chapter,book_intro_chapter,gender
0,Addam Marbrand,Lannister,NaN,NaN,NaN,56.0,1
1,Aegon Frey (Jinglebell),None,299.0,3.0,51.0,49.0,1
2,Aegon Targaryen,House Targaryen,NaN,NaN,NaN,5.0,1
3,Adrack Humble,House Greyjoy,300.0,5.0,20.0,20.0,1
4,Aemon Costayne,Lannister,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...
912,Zollo,None,NaN,NaN,NaN,21.0,1
913,Yurkhaz zo Yunzak,None,300.0,5.0,59.0,47.0,1
914,Yezzan Zo Qaggaz,None,300.0,5.0,57.0,25.0,1
915,Torwynd the Tame,Wildling,300.0,5.0,73.0,73.0,1


In [10]:
death_set.loc[death_set['name']=='Eddard Stark']

,name,allegiances,death_year,book_of_death,death_chapter,book_intro_chapter,gender
217,Eddard Stark,House Stark,299.0,1.0,65.0,1.0,1


In [11]:
characters = ["583"]

character_list = []
for character in characters:
    JSONContent = requests.get("https://anapioficeandfire.com/api/characters/" + character).json()
    if 'error' not in JSONContent:
        character_list.append([JSONContent['name'], JSONContent['gender'], JSONContent['born'],
                             JSONContent['playedBy'], JSONContent['culture'], JSONContent['aliases']])
                         
dataset = pd.DataFrame(character_list)
dataset

,0,1,2,3,4,5
0,Jon Snow,Male,In 283 AC,[Kit Harington],Northmen,"[Lord Snow, Ned Stark's Bastard, The Snow of W..."


In [12]:
r = requests.get('https://www.anapioficeandfire.com/api/characters?page=1&pageSize=10')
x = r.json()
df = pd.DataFrame(x)
display(df)

,url,name,gender,culture,born,died,titles,aliases,father,mother,spouse,allegiances,books,povBooks,tvSeries,playedBy
0,https://www.anapioficeandfire.com/api/characte...,,Female,Braavosi,,,[],[The Daughter of the Dusk],,,,[],[https://www.anapioficeandfire.com/api/books/5],[],[],[]
1,https://www.anapioficeandfire.com/api/characte...,Walder,Male,,,,[],[Hodor],,,,[https://www.anapioficeandfire.com/api/houses/...,[https://www.anapioficeandfire.com/api/books/1...,[],"[Season 1, Season 2, Season 3, Season 4, Seaso...",[Kristian Nairn]
2,https://www.anapioficeandfire.com/api/characte...,,Male,,,,[],[Lamprey],,,,[https://www.anapioficeandfire.com/api/houses/15],[https://www.anapioficeandfire.com/api/books/3],[],[],[]
3,https://www.anapioficeandfire.com/api/characte...,,Female,Braavosi,,,[],[The Merling Queen],,,,[],[https://www.anapioficeandfire.com/api/books/5...,[],[],[]
4,https://www.anapioficeandfire.com/api/characte...,,Male,,,,[],[Old Crackbones],,,,[],[https://www.anapioficeandfire.com/api/books/5],[],[],[]
5,https://www.anapioficeandfire.com/api/characte...,,Female,Braavosi,,,[],[The Poetess],,,,[],[https://www.anapioficeandfire.com/api/books/5],[],[],[]
6,https://www.anapioficeandfire.com/api/characte...,,Female,,,,[],[Porridge],,,,[https://www.anapioficeandfire.com/api/houses/15],[https://www.anapioficeandfire.com/api/books/3],[],[],[]
7,https://www.anapioficeandfire.com/api/characte...,,Male,,,,[],[Quickfinger],,,,[https://www.anapioficeandfire.com/api/houses/23],[https://www.anapioficeandfire.com/api/books/6],[],[],[]
8,https://www.anapioficeandfire.com/api/characte...,,Female,,,,[],[the Sailor's Wife],,,,[],[https://www.anapioficeandfire.com/api/books/5],[],[],[]
9,https://www.anapioficeandfire.com/api/characte...,,Female,Braavosi,,,[],[The Veiled Lady],,,,[],[https://www.anapioficeandfire.com/api/books/5],[],[],[]


In [13]:
#Since the API is deffective I will use web scraping and add the actor name for the main characters

In [14]:
# I am going to add a column with the main characters, actor names to filter out unnecessary characters and enrich the
#data set with a the an added column

In [15]:
url = "https://en.wikipedia.org/wiki/List_of_Game_of_Thrones_characters"

In [16]:
res=requests.get(url)
soup=BeautifulSoup(res.text,"html.parser")
list1=soup.select("#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child(3) > td:nth-child(1) > a")[0].text
#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child(4) > td:nth-child(1) > a
#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child(5) > td:nth-child(1) > a
list1

'Sean Bean'

In [17]:
res=requests.get(url)
soup=BeautifulSoup(res.text,"html.parser")
list1=soup.select("#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child(4) > td:nth-child(2) > a")[0].text

list1

'Robert Baratheon'

In [18]:
res=requests.get(url)
soup=BeautifulSoup(res.text,"html.parser")
list1=soup.select("#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child(3)")
list1

[<tr>
 <td><a href="/wiki/Sean_Bean" title="Sean Bean">Sean Bean</a></td>
 <td><a href="/wiki/Ned_Stark" title="Ned Stark">Eddard "Ned" Stark</a></td>
 <td class="table-cast" style="background: #ddffdd; vertical-align: middle; text-align: center;">Main</td>
 <td class="table-na" colspan="4" data-sort-value="" style="background: #ececec; color: #2C2C2C; vertical-align: middle; text-align: center;"></td>
 <td class="table-cast" style="background: #ffe3e3; vertical-align: middle; text-align: center;">Recurring<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[a]</a></sup></td>
 <td class="table-cast" style="background: #DAEBFF; vertical-align: middle; text-align: center;">Guest<sup class="reference" id="cite_ref-3"><a href="#cite_note-3">[b]</a></sup></td>
 <td class="table-na" data-sort-value="" style="background: #ececec; color: #2C2C2C; vertical-align: middle; text-align: center;">
 </td></tr>]

In [19]:
#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child(4)

In [20]:
#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child(45)

In [21]:
#Last element of list 
#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child(45) > td:nth-child(1) > a

In [22]:
character_names = []
actor_names = []

In [23]:
#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child(3) > td:nth-child(2) > a

for i in range(3,46):
    character = soup.select(f"#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child({i}) > td:nth-child(2) > a")[0].text
    character_names.append(character)



print(character_names)

['Eddard "Ned" Stark', 'Robert Baratheon', 'Jaime Lannister', 'Catelyn Stark', 'Cersei Lannister', 'Daenerys Targaryen', 'Jorah Mormont', 'Viserys Targaryen', 'Jon Snow', 'Robb Stark', 'Sansa Stark', 'Arya Stark', 'Theon Greyjoy', 'Bran Stark', 'Joffrey Baratheon', 'Sandor "The Hound" Clegane', 'Tyrion Lannister', 'Petyr "Littlefinger" Baelish', 'Davos Seaworth', 'Samwell Tarly', 'Stannis Baratheon', 'Melisandre', 'Jeor Mormont', 'Bronn', 'Varys', 'Shae', 'Margaery Tyrell', 'Tywin Lannister', 'Talisa Maegyr', 'Ygritte', 'Gendry', 'Tormund Giantsbane', 'Brienne of Tarth', 'Ramsay Bolton', 'Gilly', 'Daario Naharis', 'Missandei', 'Tommen Baratheon', 'Ellaria Sand', "Jaqen H'ghar", 'Roose Bolton', 'The High Sparrow', 'Grey Worm']


In [24]:
for i in range(3,46):
    actor = soup.select(f"#mw-content-text > div > table:nth-child(11) > tbody > tr:nth-child({i}) > td:nth-child(1) > a")[0].text
    actor_names.append(actor)

print(actor_names)

['Sean Bean', 'Mark Addy', 'Nikolaj Coster-Waldau', 'Michelle Fairley', 'Lena Headey', 'Emilia Clarke', 'Iain Glen', 'Harry Lloyd', 'Kit Harington', 'Richard Madden', 'Sophie Turner', 'Maisie Williams', 'Alfie Allen', 'Isaac Hempstead Wright', 'Jack Gleeson', 'Rory McCann', 'Peter Dinklage', 'Aidan Gillen', 'Liam Cunningham', 'John Bradley', 'Stephen Dillane', 'Carice van Houten', 'James Cosmo', 'Jerome Flynn', 'Conleth Hill', 'Sibel Kekilli', 'Natalie Dormer', 'Charles Dance', 'Oona Chaplin', 'Rose Leslie', 'Joe Dempsie', 'Kristofer Hivju', 'Gwendoline Christie', 'Iwan Rheon', 'Hannah Murray', 'Michiel Huisman', 'Nathalie Emmanuel', 'Dean-Charles Chapman', 'Indira Varma', 'Tom Wlaschiha', 'Michael McElhatton', 'Jonathan Pryce', 'Jacob Anderson']


In [25]:
death_set["actor_name"] = 0
for x in range(death_set.name.count()):
    if death_set.name[x] in character_names:
        index = character_names.index(death_set.name[x])
        death_set["actor_name"][x] = actor_names[index]
    else:
        death_set["actor_name"][x] = "Not available"

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [26]:
death_set.head()

,name,allegiances,death_year,book_of_death,death_chapter,book_intro_chapter,gender,actor_name
0,Addam Marbrand,Lannister,NaN,NaN,NaN,56.0,1,Not available
1,Aegon Frey (Jinglebell),None,299.0,3.0,51.0,49.0,1,Not available
2,Aegon Targaryen,House Targaryen,NaN,NaN,NaN,5.0,1,Not available
3,Adrack Humble,House Greyjoy,300.0,5.0,20.0,20.0,1,Not available
4,Aemon Costayne,Lannister,NaN,NaN,NaN,NaN,1,Not available


In [27]:
death_set.actor_name.value_counts()

Not available             884
Maisie Williams             1
Michiel Huisman             1
Tom Wlaschiha               1
Michael McElhatton          1
Conleth Hill                1
Harry Lloyd                 1
Emilia Clarke               1
Sibel Kekilli               1
Nikolaj Coster-Waldau       1
Richard Madden              1
Lena Headey                 1
Carice van Houten           1
Joe Dempsie                 1
Indira Varma                1
Nathalie Emmanuel           1
Iain Glen                   1
Jack Gleeson                1
Jerome Flynn                1
Alfie Allen                 1
Kit Harington               1
Hannah Murray               1
Jacob Anderson              1
Sophie Turner               1
Peter Dinklage              1
Isaac Hempstead Wright      1
Rose Leslie                 1
Charles Dance               1
John Bradley                1
Mark Addy                   1
Gwendoline Christie         1
Natalie Dormer              1
James Cosmo                 1
Liam Cunni

In [28]:
character_death_set = death_set.copy()

In [29]:
#Switching the 1s and 0s in the gender column for "Male" and female values
Male = "Male"

character_death_set['gender'] = np.where(character_death_set['gender'] == 1, Male, character_death_set['gender'])


In [32]:
Female = "Female"

character_death_set['gender'] = np.where(character_death_set['gender'] == "0", Female, character_death_set['gender'])

In [33]:
character_death_set.tail(50)

,name,allegiances,death_year,book_of_death,death_chapter,book_intro_chapter,gender,actor_name
867,Wat (sailor),None,NaN,NaN,NaN,34.0,Male,Not available
868,Watt,Night's Watch,299.0,3.0,75.0,69.0,Male,Not available
869,Watty,None,NaN,NaN,NaN,39.0,Male,Not available
870,Waymar Royce,Night's Watch,297.0,1.0,0.0,0.0,Male,Not available
871,Wayn (guard),House Stark,NaN,NaN,NaN,37.0,Male,Not available
872,Weasel,None,NaN,NaN,NaN,9.0,Female,Not available
873,Weeper,Wildling,NaN,NaN,NaN,7.0,Male,Not available
874,Weese,Lannister,299.0,2.0,38.0,26.0,Male,Not available
875,Wendel Manderly,Stark,299.0,3.0,51.0,55.0,Male,Not available
876,Wendello Qar Deeth,None,NaN,NaN,NaN,40.0,Male,Not available


In [34]:
character_death_set.to_csv('character_deaths_set.csv',index=False)